<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_T81_558/blob/master/Part2_5_Pandas_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part2.5 : Feature Engineering
特征工程是机器学习的重要组成部分。现在，我们将手工设计特征。然而，在本课程的后面，我们将看到一些自动特征工程的技术

## 1. Calculated Fields
可以将新字段添加到程序从其他字段计算出的dataframes中。 我们可以创建一个新列，以千克为单位给出重量。 给定重量（磅），计算公制重量的公式为：

$ m_{(kg)} = m_{(lb)} \times 0.45359237 $

下面的Python代码执行此转换

In [ ]:
import os
import pandas as pd
pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 5)
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])
display(df)
# 计算'weight'列的数据，并且转换为int类型
# 在添加到'weight_kg'列上
# 0：插入到第0列
df.insert(0,'weight_kg',(df['weight']*0.45359237).astype(int))
display(df)

,mpg,cylinders,displacement,...,year,origin,name
0,18.0,8,307.0,...,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,...,70,1,buick skylark 320
...,...,...,...,...,...,...,...
396,28.0,4,120.0,...,82,1,ford ranger
397,31.0,4,119.0,...,82,1,chevy s-10


,weight_kg,mpg,cylinders,...,year,origin,name
0,1589,18.0,8,...,70,1,chevrolet chevelle malibu
1,1675,15.0,8,...,70,1,buick skylark 320
...,...,...,...,...,...,...,...
396,1190,28.0,4,...,82,1,ford ranger
397,1233,31.0,4,...,82,1,chevy s-10


## 2.Google API Keys
 有时，您将使用外部API来获取数据。 以下示例显示了如何使用Google API密钥对地址进行编码以用于神经网络。 要使用它们，您将需要自己的Google API密钥。 我下面的密钥不是真实的密钥； 您需要将自己的设备放在那里。 Google会要求您提供信用卡，但是除非您使用大量查询，否则不会产生任何实际费用。 您无需获取此类的Google API密钥； 这只会告诉你如何。 如果您想获取Google API密钥，请访问此网站并获取一个地理编码。

您可以从以下链接获取自己的密钥：[Google API Keys.](https://developers.google.com/maps/documentation/embed/get-api-key)

In [ ]:
GOOGLE_KEY = '反正我是失败了，应该是没钱的原因'

## 3. 其他例子：处理地址
地址编码成神经网络是很困难的。有许多不同的方法对地址进行编码，但是必须考虑如何将地址转换成更有意义的东西。地图坐标可能是一个很好的方法。[经纬度](https://en.wikipedia.org/wiki/Geographic_coordinate_system)可能是一种有用的编码。由于互联网的强大功能，将地址转换为纬度和经度值相对容易一些。

下面的代码示例如何华盛顿大学的坐标：

In [ ]:
import requests

import requests

address = "1 Brookings Dr, St. Louis, MO 63130"

response = requests.get(
    'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}' \
    .format(GOOGLE_KEY,address))

resp_json_payload = response.json()
print(resp_json_payload)
if 'error_message' in resp_json_payload:
    print(resp_json_payload['error_message'])
else:
    print(resp_json_payload['results'][0]['geometry']['location'])


{'error_message': 'This API project is not authorized to use this API.', 'results': [], 'status': 'REQUEST_DENIED'}
This API project is not authorized to use this API.


如果经度和纬度作为两个特征输入到神经网络中，它们可能没有太大的帮助。这两个值将允许您的神经网络对地图上的位置进行聚类。有时，地图上的集群位置可能很有用。图2。SMK显示了美国各州吸烟人口的百分比。

图2：每个州的吸烟者
![Smokers by State](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_6_smokers.png "Smokers by State")

上面的地图显示，某些行为，如吸烟，可以按全球区域聚集。但是，通常您会希望将坐标转换为距离。使用球面上任意两点之间的大圆距离来估计地球上任意两点之间的距离是相当容易的:下面的代码实现了这个公式

$\Delta\sigma=\arccos\bigl(\sin\phi_1\cdot\sin\phi_2+\cos\phi_1\cdot\cos\phi_2\cdot\cos(\Delta\lambda)\bigr)$

$d = r \, \Delta\sigma$

In [ ]:
from math import sin, cos, sqrt, atan2, radians

URL='https://maps.googleapis.com' + \
    '/maps/api/geocode/json?key={}&address={}'

# Distance function
def distance_lat_lng(lat1,lng1,lat2,lng2):
    # approximate radius of earth in km
    R = 6373.0

    # degrees to radians (lat/lon are in degrees)
    lat1 = radians(lat1)
    lng1 = radians(lng1)
    lat2 = radians(lat2)
    lng2 = radians(lng2)

    dlng = lng2 - lng1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlng / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

# Find lat lon for address
def lookup_lat_lng(address):
    response = requests.get( \
        URL) \
        .format(GOOGLE_KEY,address))
    json = response.json()
    if len(json['results']) == 0:
        print("Can't find: {}".format(address))
        return 0,0
    map = json['results'][0]['geometry']['location']
    return map['lat'],map['lng']


# Distance between two locations

import requests

address1 = "1 Brookings Dr, St. Louis, MO 63130" 
address2 = "3301 College Ave, Fort Lauderdale, FL 33314"

lat1, lng1 = lookup_lat_lng(address1)
lat2, lng2 = lookup_lat_lng(address2)

print("Distance, St. Louis, MO to Ft. Lauderdale, FL: {} km".format(
        distance_lat_lng(lat1,lng1,lat2,lng2)))

SyntaxError: ignored

距离可能是编码地址的有用方法。 如果您考虑了什么距离可能对您的数据集有用，那将会有所帮助。 考虑：

* 到主要都会区的距离

* 到竞争对手的距离

* 到配送中心的距离

* 到零售店的距离

以下代码计算了10所大学与圣路易斯华盛顿大学之间的距离：

In [ ]:
# Encoding other universities by their distance to Washington University

schools = [
    ["Princeton University, Princeton, NJ 08544", 'Princeton'],
    ["Massachusetts Hall, Cambridge, MA 02138", 'Harvard'],
    ["5801 S Ellis Ave, Chicago, IL 60637", 'University of Chicago'],
    ["Yale, New Haven, CT 06520", 'Yale'],
    ["116th St & Broadway, New York, NY 10027", 'Columbia University'],
    ["450 Serra Mall, Stanford, CA 94305", 'Stanford'],
    ["77 Massachusetts Ave, Cambridge, MA 02139", 'MIT'],
    ["Duke University, Durham, NC 27708", 'Duke University'],
    ["University of Pennsylvania, Philadelphia, PA 19104", 
         'University of Pennsylvania'],
    ["Johns Hopkins University, Baltimore, MD 21218", 'Johns Hopkins']
]

lat1, lng1 = lookup_lat_lng("1 Brookings Dr, St. Louis, MO 63130")

for address, name in schools:
    lat2,lng2 = lookup_lat_lng(address)
    dist = distance_lat_lng(lat1,lng1,lat2,lng2)
    print("School '{}', distance to wustl is: {}".format(name,dist))

NameError: ignored